In [2]:
from skimage.exposure import rescale_intensity
import numpy as np
import random, os

import argparse
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import random
from matplotlib import pyplot as plt
import os, random
import numpy as np
from scipy import ndimage
from scipy import misc as sm
import cv2

In [3]:
def clamp(val, frm, to):
    if (val <= frm):
        return frm
    if (val >= to):
        return to
    return val

def apply_mat(image,i,j, mat_app):
    vr = vg = vb = 0
    clampi = i+len(mat_app)
    clampj = j+len(mat_app[0])
    
    if(clampi > len(image) or clampj > len(image[0])):
        return
    
    for c in range(i, clampi):
        for d in range(j, clampj):
            vr += image[c][d][0] * mat_app[c-i][d-j]
            vg += image[c][d][1] * mat_app[c-i][d-j]
            vb += image[c][d][2] * mat_app[c-i][d-j]
    image[i][j][0] = clamp(vr,0,255)        
    image[i][j][1] = clamp(vg,0,255)        
    image[i][j][2] = clamp(vb,0,255)
    
def apply_multichannel_mat(image, i, j, kernel_channels):
    vr = vg = vb = 0
    clampi = i+len(mat_app)
    clampj = j+len(mat_app[0])
    
    if(clampi > len(image) or clampj > len(image[0])):
        return
    
    for c in range(i, clampi):
        for d in range(j, clampj):
            vr += image[c][d][0] * kernel_channels[0][c-i][d-j]
            vg += image[c][d][1] * kernel_channels[1][c-i][d-j]
            vb += image[c][d][2] * kernel_channels[2][c-i][d-j]
    image[i][j][0] = clamp(vr,0,255)        
    image[i][j][1] = clamp(vg,0,255)        
    image[i][j][2] = clamp(vb,0,255)
    
    
def overlap(orig, other, other_opacity):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            output[i][j] = (1-other_opacity)*orig[i][j] + other_opacity*other[i][j]
    return output


def random_noise(orig, amount = 1, artifact_size = 0):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                value = np.random.rand() *output[i][j] 
                output[i][j] = value
                for x in range(i-artifact_size,i + artifact_size):    
                    if (x > 0 and x < len(orig)):
                        output[x][j] = value
                        
                for y in range(j-artifact_size,j + artifact_size):    
                    if (y > 0 and y < len(orig[0])):
                        output[i][y] = value
    return output

def salt_and_pepper(orig, amount = 1):
    output = np.copy(orig)
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                if(np.random.rand() < 0.5):
                    output[i][j] = (0,0,0)
                else:
                    output[i][j] = (1,1,1)
    return output

def swap_pixels(orig, amount = 1):
    output = np.copy(orig)
    h = len(orig)
    w = len(orig[0])
    for i in range(len(orig)):
        for j in range(len(orig[0])):
            if(np.random.rand() < amount):
                defaultp = output[i][j]
                gi, gj = np.random.randint(0,h), np.random.randint(0,w)
                output[i][j] = output[gi][gj]
                output[gi][gj] = defaultp
                
    return output

def color_int(col):
    return max(min(round(col), 255), 0)

def boxed2flat(row):
    return [_color_int(x) for box in row for x in box]

def image2file(image, path):
    """ Writes an image in list of lists format to a file. Will work with
        either color or grayscale. """
    img = image
    with open(path, 'wb') as f:
        png.Writer(width=len(image[0]), height=len(image)).write(f, [boxed2flat(r) for r in img])
        
def recursive_files(dir):
    files = []
    for r, d, f in os.walk(dir):
        for file in f:
                files.append(os.path.join(r, file))
    return files
            
def online_choice(iterable):
    for n, x in enumerate(iterable, 1):
        if random.randrange(n) == 0:
            pick = x
    return pick

In [10]:
n=1000
random.seed();

rfiles = recursive_files("./lsun/church_dataset")
print(type(list(rfiles)))
for iteration in range(n):
    rfile = rfiles[iteration]
    
    
    img = cv2.imread(rfile)
    img = resize(img, (128, 128), anti_aliasing=True)
    img = np.float32(img)
    #orig = np.copy(img)
    #random_noised  = salt_and_pepper(img, 0.03) 
    #random_noised  = random_noise(img,0.4,2)
    #random_noised  = swap_pixels(img, 0.55)
    
    #together = overlap(orig, random_noised, 1)
    gray = cv2.cvtColor(img*256, cv2.COLOR_BGR2GRAY)
    r_parts = rfile.split('/')
    r_end = r_parts[len(r_parts)-1]
                              
    no_suffix = r_end.split('.')[0]
    
    cv2.imwrite(f"./grayscale/{no_suffix}.jpg", gray)
    print(f"[{iteration+1}/{n}] {rfile} -> ", f"./grayscale/{no_suffix}.jpg")

<class 'list'>
[1/1000] ./lsun/church_dataset/0000955125.jpg ->  ./grayscale/0000955125.jpg
[2/1000] ./lsun/church_dataset/00018cf7eb.jpg ->  ./grayscale/00018cf7eb.jpg
[3/1000] ./lsun/church_dataset/0001b23b9c.jpg ->  ./grayscale/0001b23b9c.jpg
[4/1000] ./lsun/church_dataset/0002200c16.jpg ->  ./grayscale/0002200c16.jpg
[5/1000] ./lsun/church_dataset/000238fdcd.jpg ->  ./grayscale/000238fdcd.jpg
[6/1000] ./lsun/church_dataset/00024cd7b6.jpg ->  ./grayscale/00024cd7b6.jpg
[7/1000] ./lsun/church_dataset/000281b0c6.jpg ->  ./grayscale/000281b0c6.jpg
[8/1000] ./lsun/church_dataset/00029d64a9.jpg ->  ./grayscale/00029d64a9.jpg
[9/1000] ./lsun/church_dataset/00036953d8.jpg ->  ./grayscale/00036953d8.jpg
[10/1000] ./lsun/church_dataset/0004f86fef.jpg ->  ./grayscale/0004f86fef.jpg
[11/1000] ./lsun/church_dataset/00051c359a.jpg ->  ./grayscale/00051c359a.jpg
[12/1000] ./lsun/church_dataset/00052bc6a2.jpg ->  ./grayscale/00052bc6a2.jpg
[13/1000] ./lsun/church_dataset/000530cd0e.jpg ->  ./grays

[109/1000] ./lsun/church_dataset/0033af43b6.jpg ->  ./grayscale/0033af43b6.jpg
[110/1000] ./lsun/church_dataset/0033c80dc1.jpg ->  ./grayscale/0033c80dc1.jpg
[111/1000] ./lsun/church_dataset/0035db6daa.jpg ->  ./grayscale/0035db6daa.jpg
[112/1000] ./lsun/church_dataset/003677016f.jpg ->  ./grayscale/003677016f.jpg
[113/1000] ./lsun/church_dataset/0037008e0c.jpg ->  ./grayscale/0037008e0c.jpg
[114/1000] ./lsun/church_dataset/0037cad18f.jpg ->  ./grayscale/0037cad18f.jpg
[115/1000] ./lsun/church_dataset/0038309703.jpg ->  ./grayscale/0038309703.jpg
[116/1000] ./lsun/church_dataset/003917fac7.jpg ->  ./grayscale/003917fac7.jpg
[117/1000] ./lsun/church_dataset/0039a6a4f4.jpg ->  ./grayscale/0039a6a4f4.jpg
[118/1000] ./lsun/church_dataset/0039d1bd4b.jpg ->  ./grayscale/0039d1bd4b.jpg
[119/1000] ./lsun/church_dataset/003a0e4df5.jpg ->  ./grayscale/003a0e4df5.jpg
[120/1000] ./lsun/church_dataset/003a245c14.jpg ->  ./grayscale/003a245c14.jpg
[121/1000] ./lsun/church_dataset/003ac84ba9.jpg ->  

[214/1000] ./lsun/church_dataset/0065c602cd.jpg ->  ./grayscale/0065c602cd.jpg
[215/1000] ./lsun/church_dataset/0065cfa914.jpg ->  ./grayscale/0065cfa914.jpg
[216/1000] ./lsun/church_dataset/006661878b.jpg ->  ./grayscale/006661878b.jpg
[217/1000] ./lsun/church_dataset/0066842af7.jpg ->  ./grayscale/0066842af7.jpg
[218/1000] ./lsun/church_dataset/00669eef2c.jpg ->  ./grayscale/00669eef2c.jpg
[219/1000] ./lsun/church_dataset/0066aab3a2.jpg ->  ./grayscale/0066aab3a2.jpg
[220/1000] ./lsun/church_dataset/00671c7a21.jpg ->  ./grayscale/00671c7a21.jpg
[221/1000] ./lsun/church_dataset/006782dfe5.jpg ->  ./grayscale/006782dfe5.jpg
[222/1000] ./lsun/church_dataset/0067965dd2.jpg ->  ./grayscale/0067965dd2.jpg
[223/1000] ./lsun/church_dataset/0067f3d038.jpg ->  ./grayscale/0067f3d038.jpg
[224/1000] ./lsun/church_dataset/0067fc3598.jpg ->  ./grayscale/0067fc3598.jpg
[225/1000] ./lsun/church_dataset/0069688104.jpg ->  ./grayscale/0069688104.jpg
[226/1000] ./lsun/church_dataset/006ac59e0c.jpg ->  

[330/1000] ./lsun/church_dataset/00994f6dcf.jpg ->  ./grayscale/00994f6dcf.jpg
[331/1000] ./lsun/church_dataset/009acf6c59.jpg ->  ./grayscale/009acf6c59.jpg
[332/1000] ./lsun/church_dataset/009b16c2e3.jpg ->  ./grayscale/009b16c2e3.jpg
[333/1000] ./lsun/church_dataset/009c1a8f69.jpg ->  ./grayscale/009c1a8f69.jpg
[334/1000] ./lsun/church_dataset/009ca34023.jpg ->  ./grayscale/009ca34023.jpg
[335/1000] ./lsun/church_dataset/009d86a018.jpg ->  ./grayscale/009d86a018.jpg
[336/1000] ./lsun/church_dataset/009dea7834.jpg ->  ./grayscale/009dea7834.jpg
[337/1000] ./lsun/church_dataset/009e351309.jpg ->  ./grayscale/009e351309.jpg
[338/1000] ./lsun/church_dataset/009f1ecd15.jpg ->  ./grayscale/009f1ecd15.jpg
[339/1000] ./lsun/church_dataset/009f6cfd02.jpg ->  ./grayscale/009f6cfd02.jpg
[340/1000] ./lsun/church_dataset/00a2b214c2.jpg ->  ./grayscale/00a2b214c2.jpg
[341/1000] ./lsun/church_dataset/00a3d0c9f5.jpg ->  ./grayscale/00a3d0c9f5.jpg
[342/1000] ./lsun/church_dataset/00a4543840.jpg ->  

[446/1000] ./lsun/church_dataset/00d7af06e7.jpg ->  ./grayscale/00d7af06e7.jpg
[447/1000] ./lsun/church_dataset/00d8b3cc8e.jpg ->  ./grayscale/00d8b3cc8e.jpg
[448/1000] ./lsun/church_dataset/00d8b678a3.jpg ->  ./grayscale/00d8b678a3.jpg
[449/1000] ./lsun/church_dataset/00d9489735.jpg ->  ./grayscale/00d9489735.jpg
[450/1000] ./lsun/church_dataset/00d94bf9a0.jpg ->  ./grayscale/00d94bf9a0.jpg
[451/1000] ./lsun/church_dataset/00da21225e.jpg ->  ./grayscale/00da21225e.jpg
[452/1000] ./lsun/church_dataset/00da65a1f7.jpg ->  ./grayscale/00da65a1f7.jpg
[453/1000] ./lsun/church_dataset/00da698d3f.jpg ->  ./grayscale/00da698d3f.jpg
[454/1000] ./lsun/church_dataset/00db0f847e.jpg ->  ./grayscale/00db0f847e.jpg
[455/1000] ./lsun/church_dataset/00db667512.jpg ->  ./grayscale/00db667512.jpg
[456/1000] ./lsun/church_dataset/00dbb96dfb.jpg ->  ./grayscale/00dbb96dfb.jpg
[457/1000] ./lsun/church_dataset/00dbc3b1f5.jpg ->  ./grayscale/00dbc3b1f5.jpg
[458/1000] ./lsun/church_dataset/00dbdad3d0.jpg ->  

[552/1000] ./lsun/church_dataset/010db38072.jpg ->  ./grayscale/010db38072.jpg
[553/1000] ./lsun/church_dataset/010dc95e49.jpg ->  ./grayscale/010dc95e49.jpg
[554/1000] ./lsun/church_dataset/010ed8aeee.jpg ->  ./grayscale/010ed8aeee.jpg
[555/1000] ./lsun/church_dataset/010f971b9a.jpg ->  ./grayscale/010f971b9a.jpg
[556/1000] ./lsun/church_dataset/011250ee6f.jpg ->  ./grayscale/011250ee6f.jpg
[557/1000] ./lsun/church_dataset/01125cc700.jpg ->  ./grayscale/01125cc700.jpg
[558/1000] ./lsun/church_dataset/01129cd290.jpg ->  ./grayscale/01129cd290.jpg
[559/1000] ./lsun/church_dataset/0112e2a3e2.jpg ->  ./grayscale/0112e2a3e2.jpg
[560/1000] ./lsun/church_dataset/011346884d.jpg ->  ./grayscale/011346884d.jpg
[561/1000] ./lsun/church_dataset/01136fd671.jpg ->  ./grayscale/01136fd671.jpg
[562/1000] ./lsun/church_dataset/0113bbbecd.jpg ->  ./grayscale/0113bbbecd.jpg
[563/1000] ./lsun/church_dataset/01145da7c2.jpg ->  ./grayscale/01145da7c2.jpg
[564/1000] ./lsun/church_dataset/011503b39e.jpg ->  

[659/1000] ./lsun/church_dataset/0146d8effa.jpg ->  ./grayscale/0146d8effa.jpg
[660/1000] ./lsun/church_dataset/0147a6d6df.jpg ->  ./grayscale/0147a6d6df.jpg
[661/1000] ./lsun/church_dataset/0147a8fe68.jpg ->  ./grayscale/0147a8fe68.jpg
[662/1000] ./lsun/church_dataset/01486ac8f7.jpg ->  ./grayscale/01486ac8f7.jpg
[663/1000] ./lsun/church_dataset/0148e1d893.jpg ->  ./grayscale/0148e1d893.jpg
[664/1000] ./lsun/church_dataset/0148f9a53c.jpg ->  ./grayscale/0148f9a53c.jpg
[665/1000] ./lsun/church_dataset/0149f5554b.jpg ->  ./grayscale/0149f5554b.jpg
[666/1000] ./lsun/church_dataset/014aff4490.jpg ->  ./grayscale/014aff4490.jpg
[667/1000] ./lsun/church_dataset/014b4b9f4b.jpg ->  ./grayscale/014b4b9f4b.jpg
[668/1000] ./lsun/church_dataset/014b93ecec.jpg ->  ./grayscale/014b93ecec.jpg
[669/1000] ./lsun/church_dataset/014d35ea62.jpg ->  ./grayscale/014d35ea62.jpg
[670/1000] ./lsun/church_dataset/014d5465f1.jpg ->  ./grayscale/014d5465f1.jpg
[671/1000] ./lsun/church_dataset/014d6389de.jpg ->  

[764/1000] ./lsun/church_dataset/0182d51598.jpg ->  ./grayscale/0182d51598.jpg
[765/1000] ./lsun/church_dataset/0182db64d1.jpg ->  ./grayscale/0182db64d1.jpg
[766/1000] ./lsun/church_dataset/0184a52193.jpg ->  ./grayscale/0184a52193.jpg
[767/1000] ./lsun/church_dataset/0184c9e814.jpg ->  ./grayscale/0184c9e814.jpg
[768/1000] ./lsun/church_dataset/01857f7a25.jpg ->  ./grayscale/01857f7a25.jpg
[769/1000] ./lsun/church_dataset/0185a83bb3.jpg ->  ./grayscale/0185a83bb3.jpg
[770/1000] ./lsun/church_dataset/0185b81a86.jpg ->  ./grayscale/0185b81a86.jpg
[771/1000] ./lsun/church_dataset/0186df5cb0.jpg ->  ./grayscale/0186df5cb0.jpg
[772/1000] ./lsun/church_dataset/0188070563.jpg ->  ./grayscale/0188070563.jpg
[773/1000] ./lsun/church_dataset/0188152e82.jpg ->  ./grayscale/0188152e82.jpg
[774/1000] ./lsun/church_dataset/018887cfcb.jpg ->  ./grayscale/018887cfcb.jpg
[775/1000] ./lsun/church_dataset/0188c646ee.jpg ->  ./grayscale/0188c646ee.jpg
[776/1000] ./lsun/church_dataset/0189183d54.jpg ->  

[872/1000] ./lsun/church_dataset/01b8cdb018.jpg ->  ./grayscale/01b8cdb018.jpg
[873/1000] ./lsun/church_dataset/01b9656eea.jpg ->  ./grayscale/01b9656eea.jpg
[874/1000] ./lsun/church_dataset/01b978682c.jpg ->  ./grayscale/01b978682c.jpg
[875/1000] ./lsun/church_dataset/01b9c7f76c.jpg ->  ./grayscale/01b9c7f76c.jpg
[876/1000] ./lsun/church_dataset/01b9cbe250.jpg ->  ./grayscale/01b9cbe250.jpg
[877/1000] ./lsun/church_dataset/01b9f263c5.jpg ->  ./grayscale/01b9f263c5.jpg
[878/1000] ./lsun/church_dataset/01bb48a15a.jpg ->  ./grayscale/01bb48a15a.jpg
[879/1000] ./lsun/church_dataset/01bbb82099.jpg ->  ./grayscale/01bbb82099.jpg
[880/1000] ./lsun/church_dataset/01bbf5f01e.jpg ->  ./grayscale/01bbf5f01e.jpg
[881/1000] ./lsun/church_dataset/01bc008d98.jpg ->  ./grayscale/01bc008d98.jpg
[882/1000] ./lsun/church_dataset/01bc7f9be3.jpg ->  ./grayscale/01bc7f9be3.jpg
[883/1000] ./lsun/church_dataset/01bd55aa16.jpg ->  ./grayscale/01bd55aa16.jpg
[884/1000] ./lsun/church_dataset/01bd65a02a.jpg ->  

[979/1000] ./lsun/church_dataset/01ed49f5ab.jpg ->  ./grayscale/01ed49f5ab.jpg
[980/1000] ./lsun/church_dataset/01ed5cbe7d.jpg ->  ./grayscale/01ed5cbe7d.jpg
[981/1000] ./lsun/church_dataset/01ed65fc7d.jpg ->  ./grayscale/01ed65fc7d.jpg
[982/1000] ./lsun/church_dataset/01ef2933bc.jpg ->  ./grayscale/01ef2933bc.jpg
[983/1000] ./lsun/church_dataset/01f02f2b81.jpg ->  ./grayscale/01f02f2b81.jpg
[984/1000] ./lsun/church_dataset/01f15936c3.jpg ->  ./grayscale/01f15936c3.jpg
[985/1000] ./lsun/church_dataset/01f1c36df2.jpg ->  ./grayscale/01f1c36df2.jpg
[986/1000] ./lsun/church_dataset/01f33ad0f9.jpg ->  ./grayscale/01f33ad0f9.jpg
[987/1000] ./lsun/church_dataset/01f371e213.jpg ->  ./grayscale/01f371e213.jpg
[988/1000] ./lsun/church_dataset/01f41dcdf6.jpg ->  ./grayscale/01f41dcdf6.jpg
[989/1000] ./lsun/church_dataset/01f499a461.jpg ->  ./grayscale/01f499a461.jpg
[990/1000] ./lsun/church_dataset/01f4cd4bde.jpg ->  ./grayscale/01f4cd4bde.jpg
[991/1000] ./lsun/church_dataset/01f507d08b.jpg ->  